In [ ]:
#install minimint (https://github.com/segasai/minimint) with pip install minimint, on first import run minimint.download_and_prepare()

import xymass
import numpy as np
import minimint
import astropy
import matplotlib.pyplot as plt
import astropy.units as u
import scipy
from isochrones.mist import MIST_Isochrone
from isochrones.mist import MIST_EvolutionTrack
from isochrones.mist.bc import MISTBolometricCorrectionGrid
from isochrones.mist import MISTEvolutionTrackGrid, MISTIsochroneGrid

In [ ]:
logage=np.log10(1.e+9) # log10(age/year) of stellar population
feh=-2. #[Fe/H] metallicity of stellar population
distance=100000.*u.pc #distance to stellar population
dmodulus=5.*np.log10(distance.to(u.pc).value)-5.
resolution_limit=0.5*u.arcsec #instrument angular resolution limit
resolution_limit_physical=resolution_limit.to(u.radian).value*distance

In [ ]:
ii=minimint.Interpolator(['Gaia_G_EDR3']) #filter doesn't matter here, just needed to pick one
max_mass=ii.getMaxMass(logage,feh) #maximum mass, in M_sun, for adopted age, metallicity

In [ ]:
n_object=1000000 #sample size

sb_model='plum' #specify surface brightness model ('plum', 'exp', '2bg', 'uni')
r_scale=100.*u.pc #specify scale radius
ellipticity=0.5 #specify ellipticity for flattened (elliptical) system, e=1-b/a
position_angle=55. #specify position angle for flattened system

#generate sample of stellar positions according to parameters above
r2d=xymass.sample_r2d(size=n_object,model=sb_model,r_scale=r_scale,ellipticity=ellipticity,position_angle=position_angle) 

In [ ]:
m_min=0.1 #specify minimum stellar mass in M_sun
imf_model='kroupa' #specify IMF model ('kroupa', 'salpeter', 'BPL', 'lognormal')
imf_m_min=0.1 #specify minimum stellar mass in M_sun
mass_primary=xymass.sample_imf(size=n_object,model=imf_model,m_min=imf_m_min,m_max=max_mass).mass*u.M_sun


In [ ]:
binary_model='Raghavan2010'
f_binary=0.2 #specify binary fraction.  Expected number of stars will become n_object*(1+f_binary).  
#Actual number will be Poisson distributed

#randomly (according to f_binary) select objects from r2d to be replaced by two components comprising binary system
#sample positions of binary companions from physically-calculated orbits according to binary_model
r2d_wb=xymass.add_binaries(r2d.r_xyz,mass_primary,f_binary=f_binary,\
                                               m_min=m_min,binary_model=binary_model)

singles=np.where(r2d_wb.item==0)[0] #single objects
primaries=np.where(r2d_wb.item==1)[0] #primaries within binary objects
secondaries=np.where(r2d_wb.item==2)[0] #secondaries within binary objects

In [ ]:
#need to make sure these filters are in bc.py file in site-packages
filter_names=['G','BP','RP','DECam_g','DECam_r','DECam_i','DECam_z','DECam_Y','SDSS_u','SDSS_g','SDSS_r','SDSS_i','SDSS_z','PS_g','PS_r','PS_i','PS_z','V','F090W','F150W','ACS_WFC_F606W','ACS_WFC_F814W','LSST_u','LSST_g','LSST_r','LSST_i','LSST_z','LSST_y','R062','Z087','Y106','J129','W146','H158','F184']

mist_track=MIST_EvolutionTrack()
bc_grid=MISTBolometricCorrectionGrid(filter_names)

In [ ]:
#interpolate isochrone at each mass
iso=mist_track.generate(r2d_wb.mass,logage,feh,accurate=True)
#get bolometric correction for all filters in filter_names
iso_bc=bc_grid.interp([iso['Teff'],iso['logg'],iso['feh'],np.zeros(len(iso))],filter_names)
#apply bolometric corrections
iso_filter_mag=(np.array(iso['Mbol'])-iso_bc.T).T

In [ ]:
Mbol_sun=4.74
r2d_with_binaries_blend=xymass.binary_blend(r2d_wb,iso_filter_mag,Mbol_sun,resolution_limit_physical)

In [ ]:
L_V=10.**(-(iso_filter_mag[17]-Mbol_sun)/2.5)
M_V=Mbol_sun-2.5*np.log10(np.sum(L_V))
mass_tot=np.sum(r2d_wb.mass)

f090w=r2d_with_binaries_blend.mag.T[18]+dmodulus
f150w=r2d_with_binaries_blend.mag.T[19]+dmodulus

#simulate observational errors similar to JWST program
e_f090w=10.**(0.3*f090w-9.7)
e_f150w=10.**(0.3*f150w-9.35)

f090w_obs=f090w+np.random.normal(loc=0.,scale=e_f090w)
f150w_obs=f150w+np.random.normal(loc=0.,scale=e_f150w)

mag=f090w_obs
col=f090w_obs-f150w_obs

In [ ]:
fig=plt.figure(1)
ax1=fig.add_subplot(221)
ax2=fig.add_subplot(222)
ax3=fig.add_subplot(223)
ax4=fig.add_subplot(224)
fig.subplots_adjust(wspace=0.5,hspace=0.5)

cols=np.full(len(r2d_wb.item),'k')
cols[r2d_wb.item>0]='r'
ax1.scatter([-999],[-999],color='k',label='single',s=1)
ax1.scatter([-999],[-999],color='r',label='binary',s=1)
ax1.scatter(r2d_wb.r_xyz.T[0]/r2d.rhalf_2d.value,r2d_wb.r_xyz.T[1]/r2d.rhalf_2d.value,color=cols,s=1,alpha=0.3)
ax1.set_xlabel(r'$X / R_{\rm half}$')
ax1.set_ylabel(r'$Y / R_{\rm half}$')
ax1.legend(loc=1,fontsize=6)
ax1.set_xlim([-10,10])
ax1.set_ylim([-10,10])
ax1.set_aspect('equal')

#ax2.hist(np.log10(nn/r2d.rhalf_2d.value),histtype='step',color='k',label='all',bins=50)
#ax2.hist(np.log10(nn_single/r2d.rhalf_2d.value),histtype='step',color='b',label='single',bins=50)
#ax2.hist(np.log10(nn_binary/r2d.rhalf_2d.value),histtype='step',color='r',label='binary',bins=50)
#ax2.set_xlabel(r'$\log_{10}[\mathrm{NN sep}/R_{\rm half}]$')
#ax2.set_ylabel('N')
#ax2.legend(loc=2,fontsize=6)

cols=np.full(len(col),'k',dtype='str')
cols[r2d_with_binaries_blend.blend]='r'
ax3.scatter([-999],[-999],color='k',label='single source',s=1)
ax3.scatter([-999],[-999],color='r',label='blend',s=1)
ax3.scatter(col,mag,color=cols,alpha=0.3,s=1)
ax3.set_xlabel('F090W -- F150W')
ax3.set_ylabel('F090W')
ax3.set_ylim([30,16])
ax3.set_xlim([-1,2])
ax3.legend(fontsize=6)

ax4.set_xlim([0,1])
ax4.set_ylim([0,1])
ax4.text(0.03,0.9,'SB model: '+sb_model+r', $R_{\rm scale}=$'+str(round(r_scale.value,0))+' '+str(r_scale.unit),fontsize=7)
ax4.text(0.03,0.78,r' $1-b/a = $'+str(round(ellipticity,2,))+r', $\theta=$'+str(round(position_angle,2))+r'$^{\circ}$',fontsize=7)
ax4.text(0.03,0.66,'[Fe/H]='+str(feh)+r', $\log_{10}[\mathrm{age/yr}]=$'+str(round(logage,2)),fontsize=7)
ax4.text(0.03,0.54,'Distance = '+str(round(distance.value,0))+' kpc',fontsize=7)
ax4.text(0.03,0.42,'IMF: '+imf_model+', '+r'$(m_{\rm min},m_{\rm max})=($'+str(round(imf_m_min,2))+','+str(round(max_mass,2))+'$)$',fontsize=7)
ax4.text(0.03,0.30,r'$M_V=$'+str(round(M_V,2))+', '+r'$\log_{10}[M_*/M_{\odot}]=$'+str(round(np.log10(mass_tot),2)),fontsize=7)
ax4.text(0.03,0.18,'Binaries: '+binary_model+', '+r'$f_{\rm b}=$'+str(round(f_binary,2)),fontsize=7)
ax4.text(0.03,0.06,'Resolution limit: '+str(round(resolution_limit.value,2))+' arcsec',fontsize=7)
ax4.tick_params(left=False,top=False,bottom=False,right=False,labelleft=False,labeltop=False,labelbottom=False,labelright=False)
plt.show()
